In [19]:
import yaml
import pandas as pd 
controls = ['negative_control','positive_control']
grn_models = ['collectri','granie', 'figr', 'celloracle', 'scglue', 'scenicplus']

# Robustness analysis

In [47]:
!viash ns build --setup cb --parallel
# !viash ns build --setup cb -q run_robustness_analysis

temporaryFolder: /tmp/viash_hub_repo12671172282063304508 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
temporaryFolder: /tmp/viash_hub_repo14261181966255345651 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
Exporting format_resources_r (multiomics) =docker=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/docker/multiomics/format_resources_r
Exporting batch_correction_seurat (perturbation) =native=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/native/perturbation/batch_correction_seurat
Exporting sc_counts (perturbation) =nextflow=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/nextflow/perturbation/sc_counts
Exporting regression_2 (metrics) =nextflow=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn

## Shuffle net

In [40]:
!bash scripts/run_robust_analys.sh "net"

net
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [astonishing_poincare] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

[-        ] process > run_robustness_analysis:run... [  0%] 0 of 4
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[4b/08b1c4] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run...

In [63]:
base_folder = 'output/robust_analy_net'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, degree, method = method_id.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(degree), method] = ex_true_tf_value

df_reg1.sort_index()


,collectri,granie,figr,celloracle,scglue,scenicplus
0,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
10,-0.315255,0.170255,0.139809,0.203669,0.132937,0.312483
20,-0.369273,0.160628,0.125023,0.183348,0.156769,0.293282
50,-0.421345,0.105859,0.067059,0.110983,0.081232,0.231999
100,-0.456039,-0.028658,-0.090232,-0.003224,-0.10102,-0.018023


## Add noise to weights

In [41]:
!bash scripts/run_robust_analys.sh "weight"

weight
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [dreamy_davinci] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (1)
[5f/aa4675] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f3/689356] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustn

In [43]:
base_folder = 'output/robust_analy_weight'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, degree, method = method_id.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(degree), method] = ex_true_tf_value

df_reg1.sort_index()

,collectri,granie,figr,celloracle,scglue,scenicplus
0,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
10,-0.224716,0.185143,0.18504,0.227575,0.225391,0.333008
20,-0.228165,0.183382,0.180036,0.224755,0.221201,0.332981
50,-0.235282,0.174765,0.164224,0.20491,0.167678,0.325849
100,-0.297882,0.146087,0.117188,0.167833,0.120502,0.304062


## Negative vs positive controls

In [36]:
!bash scripts/run_pc_vs_nc.sh "pc_vs_nc"

Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [magical_mercator] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (2)
[c6/b7f5d5] process > run_robustness_analysis:run... [  0%] 0 of 18
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[4f/bc6528] process > run_robustness_analysis:run... [  0%] 0 of 18
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_a

In [39]:
base_folder = 'output/robust_analy_pc_vs_nc'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, rest = method_id.split('--')
    subsample, method = rest.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(subsample), method] = ex_true_tf_value

df_reg1 = df_reg1.sort_index()
df_reg1.index = ['Default', 'Negative control', 'Positive control']
df_reg1

,collectri,granie,figr,celloracle,scglue,scenicplus
Default,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
Negative control,-0.249304,0.200482,0.211147,0.234089,0.250881,0.358298
Positive control,-0.242072,0.175922,0.179734,0.215799,0.218924,0.316465


## regulatory sign

In [50]:
!bash scripts/run_robust_analys.sh "sign"

sign
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [soggy_sinoussi] DSL2 - revision: d55e1b9116
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (2)
[2f/3f5479] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f9/652d7d] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustne

In [60]:
base_folder = 'output/robust_analy_sign'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, degree, method = method_id.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(degree), method] = ex_true_tf_value

df_reg1 = df_reg1.sort_index()
df_reg1 = df_reg1.drop(100)
df_reg1.index = ['Degree 0%', 'Degree 10%', 'Degree 20%', 'Degree 50%']

df_reg1

,collectri,granie,figr,celloracle,scglue,scenicplus
Degree 0%,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
Degree 10%,-0.274131,0.121877,0.150885,0.170756,0.182001,0.288634
Degree 20%,-0.288105,0.083203,0.093822,0.106046,0.130437,0.215896
Degree 50%,-0.333019,-0.008502,-0.037703,-0.009838,-0.061216,-0.009351


## Corr vs causal

In [13]:
import anndata as ad 
import numpy as np
import pandas as pd
import anndata as ad
import sys
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

par = {
  "perturbation_data": "resources/grn-benchmark/perturbation_data.h5ad",
  "layer": "pearson",
  "tf_all":  "resources/prior/tf_all.csv"
}
def create_positive_control(X: np.ndarray, groups: np.ndarray):
    grns = []
    for group in tqdm(np.unique(groups), desc="Processing groups"):
        X_sub = X[groups == group, :]
        X_sub = StandardScaler().fit_transform(X_sub)
        grn = np.dot(X_sub.T, X_sub) / X_sub.shape[0]
        grns.append(grn)
    return np.mean(grns, axis=0)
perturbation_data = ad.read_h5ad(par["perturbation_data"])
gene_names = perturbation_data.var_names.to_numpy()
tf_all = np.loadtxt(par['tf_all'], dtype=str)
groups = perturbation_data.obs.cell_type
tf_all = np.intersect1d(tf_all, gene_names)

In [30]:
net = create_positive_control(perturbation_data.layers[par["layer"]], groups)
net = pd.DataFrame(net, index=gene_names, columns=gene_names)
# subset for genes
# n_genes = 10000
# net = net.sample(n_genes)
# subset for tfs
# n_central_genes = 10000
# central_tfs = net.abs().sum(axis=0).argsort()[::-1][:n_central_genes].index.to_numpy()
central_tfs = net.abs().sum(axis=0).argsort()[::-1].index.to_numpy()
net = net[central_tfs]

Processing groups: 100%|██████████| 4/4 [00:08<00:00,  2.23s/it]


In [31]:
tfs = list(set(tf_all) & set(central_tfs))

In [32]:
n_iter = 10
# net_corr = net.sample(len(tfs), axis=1)
# net_corr

In [33]:
for i in range(n_iter):
    net_corr = net.sample(len(tfs), axis=1)
    net_corr = net_corr.reset_index().melt(id_vars='index', var_name='source', value_name='weight')
    net_corr.rename(columns={'index': 'target'}, inplace=True)
    net_corr.to_csv(f'output/causal/grns/net_corr_{i}.csv')

In [34]:
net_pc = net[tfs]
net_pc = net_pc.reset_index().melt(id_vars='index', var_name='source', value_name='weight')
net_pc.rename(columns={'index': 'target'}, inplace=True)
net_pc.to_csv('output/causal/grns/net_pc.csv')

In [35]:
import subprocess

for i in range(1, n_iter+1):
    cmd = f"viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/causal/grns/net_corr_{i}.csv --score output/causal/scores/net_corr_{i}.h5ad"
    subprocess.run(cmd, shell=True)


Reading input files
(549, 15215)
Compute metrics for layer: pearson
ex(False)_tf(-1)

Processing groups: 100%|██████████| 5/5 [00:02<00:00,  2.31it/s]
/viash_automount/tmp/viash-run-regression_1-uTh16s.py:47: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  output = ad.AnnData(
   ex(False)_tf(-1)      Mean
0          0.400691  0.400691
(2,) (2,)
Reading input files
(549, 15215)
Compute metrics for layer: pearson
ex(False)_tf(-1)

Processing groups: 100%|██████████| 5/5 [00:02<00:00,  2.22it/s]
/viash_automount/tmp/viash-run-regression_1-nG5R5y.py:47: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future b

In [36]:
cmd = f"viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/causal/grns/net_pc.csv --score output/causal/scores/net_pc.h5ad"
subprocess.run(cmd, shell=True)

Reading input files
(549, 15215)
Compute metrics for layer: pearson
ex(False)_tf(-1)

Processing groups: 100%|██████████| 5/5 [00:02<00:00,  2.38it/s]
/viash_automount/tmp/viash-run-regression_1-8IIAE7.py:47: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  output = ad.AnnData(
   ex(False)_tf(-1)      Mean
0          0.401503  0.401503
(2,) (2,)


CompletedProcess(args='viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/causal/grns/net_pc.csv --score output/causal/scores/net_pc.h5ad', returncode=0)

In [37]:
# 
corr_scores = []
for i in range(1, n_iter):    
    corr_scores.append(ad.read_h5ad(f"output/causal/scores/net_corr_{i}.h5ad").uns['metric_values'][0])
pc_score = ad.read_h5ad(f"output/causal/scores/net_pc.h5ad").uns['metric_values'][0]

In [38]:

corr_scores

[0.4006913500416017,
 0.4017435476729577,
 0.3981992586017887,
 0.39803022524936194,
 0.4023016432251096,
 0.39771961848119314,
 0.3965104014168827,
 0.39915808837861566,
 0.4005040771341226]

In [39]:
pc_score

0.40150344303057306

In [40]:
(corr_scores<pc_score).sum()

7

In [41]:
import pandas as pd 
pd.read_csv('resources_test/prior/cell_topic.csv', index_col=0).index

Index(['AAGCCACGTCAGGCAT3', 'AAGGATGTCAGGCTAT3', 'AAGTTACGTTAGGCTA3',
       'ACCTGGATCCCTCAGT3', 'ACCTTGTGTACTTCAC3', 'AGACTATGTGCGCGTA3',
       'AGAGAGGAGCTTAACA3', 'AGCTATATCATGTCAA3', 'AGCTTGGTCCTAGTTT3',
       'AGTAGCTTCCTTAGGG3',
       ...
       'CTTACTAGTGCCTCAC3', 'AACTAGCTCTGGCAAT3', 'TCCTCTAAGGCGCTAC3',
       'TCATGTTTCGTTACTT3', 'ACTTGAATCAGCACGC3', 'TAGTGTGGTTGGTTGA3',
       'GTCATGCCACCTCGCT3', 'AGTAAACAGTCACCAG3', 'ACCTCACTCAAGGACA3',
       'CTTAACAAGAGCCGGA3'],
      dtype='object', length=2000)

In [42]:
ad.read_h5ad('resources_test/grn-benchmark/multiomics_rna.h5ad').obs.index.str.replace('-','')

Index(['GTCTTTAGTTAGTGAT3', 'TAATGCATCGAGGAGT3', 'TCTAGCACACTATGGC3',
       'GCTAATATCGTCAAGT3', 'GTTACGTAGGTGCGGA3', 'CATTCCTCATTAAGTC3',
       'GGGTGTTGTTACTTGC3', 'GCAAGCCTCTAAGTCA3', 'CAATCTAAGGCTACAT3',
       'AAGCGAGGTTGTTGCT3',
       ...
       'GTGCTTACATGGCCCA3', 'ACGTTACAGCGAGCGA3', 'TGAGCACGTTGCCTCA3',
       'CTTAATGAGTTTCCTG3', 'AGGTGAATCGAGGAAC3', 'TCATGAGGTTTGTTGC3',
       'AAGTGAAGTCTTACTA3', 'ATACCTGCAGGAACTG3', 'CGCAATAGTGTAATAC3',
       'CTGCAATAGGTCCAAT3'],
      dtype='object', name='obs_id', length=2000)

In [43]:
import anndata as ad 
ad.read_h5ad('resources_test/grn-benchmark/multiomics_rna.h5ad').obs.index.str.replace('-','').isin(pd.read_csv('resources_test/prior/cell_topic.csv', index_col=0).index).sum()

2000

In [44]:
cell_topic=pd.read_csv('output/scenicplus/cell_topic.csv', index_col=0)
cell_topic.index = cell_topic.index.str.split('-').str[0]
cell_topic.to_csv('output/scenicplus/cell_topic.csv')